<a href="https://colab.research.google.com/github/Dharunika-07/Political_sentiment_analysis/blob/main/Sentiment_analysis_LSTM.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
pip install tensorflow keras numpy pandas matplotlib


In [2]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Embedding, LSTM, Dense, Dropout, SpatialDropout1D
from tensorflow.keras.preprocessing.text import Tokenizer
from tensorflow.keras.preprocessing.sequence import pad_sequences
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import LabelEncoder

In [17]:
from google.colab import drive
drive.mount('/content/drive')


Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [5]:
train_df = pd.read_csv("/content/drive/MyDrive/PS_train.csv")  # Replace with your train file
test_df = pd.read_csv("/content/drive/MyDrive/PS_test_without_lables.csv")  # Replace with your test file
print(train_df.head())
print(test_df.head())


                                             content         labels
0  தென்காசி தொகுதி புதிய தமிழகம் கட்சி வேட்பாளர் ...        Neutral
1  அண்ணன் இதனை சூசகமாக 11 மாதங்கள் முன்பே பேட்டிய...  Substantiated
2  ஒரு வருடம் ஆகி விட்டது இந்த துயரம் நேர்ந்து......    Opinionated
3  எடப்பாடியை கண்டுகொள்ளாத "எடப்பாடி"🫢\n ---\nஆதர...       Positive
4  எங்களின் அரசியல் அடுத்த தலைமுறைக்குமானது \n#மக...    Opinionated
      Id                                            content
0  PS_01  இஸ்லாமிய சகோதரர்களுடன் ரமலான் கொண்டாடிய அதிமுக...
1  PS_02  \nஓபிஎஸ் - எடப்பாடி போட்டா போட்டி! திடீரென பணி...
2  PS_03  இன்றைய பரப்புரை:\n\nநாம் தமிழர் கட்சி தலைமை ஒர...
3  PS_04  🇰🇬🎙️ இன்னும் 05 ஏ நாளில், வெல்வோம் தமிழராய் - ...
4  PS_05  டாஸ்மாக்ல மட்டும்தான் கருணாநிதி அவர்களின்  பெய...


In [6]:
X = train_df['content']  # Replace 'text' with the column containing Tamil text
y = train_df['labels']  # Replace 'label' with the column containing sentiment labels

# Encode labels
encoder = LabelEncoder()
y = encoder.fit_transform(y)
print(train_df['labels'].head(30))
print("y: ",y[0:30])
# Split into training and validation sets
X_train, X_val, y_train, y_val = train_test_split(X, y, test_size=0.2, random_state=42)

0               Neutral
1         Substantiated
2           Opinionated
3              Positive
4           Opinionated
5         Substantiated
6         Substantiated
7              Positive
8              Positive
9           Opinionated
10          Opinionated
11            Sarcastic
12          Opinionated
13             Positive
14            Sarcastic
15             Negative
16              Neutral
17            Sarcastic
18        Substantiated
19              Neutral
20              Neutral
21    None of the above
22          Opinionated
23              Neutral
24          Opinionated
25        Substantiated
26             Negative
27            Sarcastic
28            Sarcastic
29            Sarcastic
Name: labels, dtype: object
y:  [1 6 3 4 3 6 6 4 4 3 3 5 3 4 5 0 1 5 6 1 1 2 3 1 3 6 0 5 5 5]


In [7]:
tokenizer = Tokenizer(num_words=5000, oov_token="<OOV>")
tokenizer.fit_on_texts(X_train)

X_train_seq = tokenizer.texts_to_sequences(X_train)
X_test_seq = tokenizer.texts_to_sequences(X_val)

max_len = 50  # Maximum length of sequences
X_train_pad = pad_sequences(X_train_seq, maxlen=max_len, padding='post', truncating='post')
X_test_pad = pad_sequences(X_test_seq, maxlen=max_len, padding='post', truncating='post')


In [8]:
model = Sequential([
    Embedding(input_dim=5000, output_dim=128, input_length=max_len),
    SpatialDropout1D(0.2),
    LSTM(128, dropout=0.2, recurrent_dropout=0.2),
    Dense(64, activation='relu'),
    Dropout(0.5),
    Dense(3, activation='softmax')  # 3 classes: Negative, Neutral, Positive
])

model.compile(
    loss='sparse_categorical_crossentropy',
    optimizer='adam',
    metrics=['accuracy']
)

model.summary()


/usr/local/lib/python3.11/dist-packages/keras/src/layers/core/embedding.py:90: UserWarning: Argument `input_length` is deprecated. Just remove it.
  warnings.warn(


Model: "sequential"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━┓
┃ Layer (type)                         ┃ Output Shape                ┃         Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━┩
│ embedding (Embedding)                │ ?                           │     0 (unbuilt) │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ spatial_dropout1d (SpatialDropout1D) │ ?                           │               0 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ lstm (LSTM)                          │ ?                           │     0 (unbuilt) │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ dense (Dense)                        │ ?                           │     0 (unbuilt) │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ dropout (Dropout)                    │ ?                           │               0 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ dense_1 (Dense)                      │ ?                           │     0 (unbuilt) │
└──────────────────────────────────────┴─────────────────────────────┴─────────────────┘

 Total params: 0 (0.00 B)

 Trainable params: 0 (0.00 B)

 Non-trainable params: 0 (0.00 B)

In [9]:
y_val = y_val.reshape(-1)
print("y_val shape after reshape:", y_val.shape)

y_val shape after reshape: (871,)


In [10]:
import numpy as np

# Convert to NumPy arrays if needed
X_test_pad = np.array(X_test_pad)
y_val = np.array(y_val)

# Check data types
print("X_test_pad dtype:", X_test_pad.dtype)
print("y_val dtype:", y_val.dtype)


X_test_pad dtype: int32
y_val dtype: int64


In [11]:
if len(y_val.shape) > 1:
    y_val = y_val.reshape(-1)  # Flatten to 1D
print("y_val shape:", y_val.shape)

from tensorflow.keras.preprocessing.sequence import pad_sequences

# Ensure consistent padding
X_test_pad = pad_sequences(X_test_pad, maxlen=50, padding='post', truncating='post')


y_val shape: (871,)


In [12]:
print("Unique values in y_train:", set(y_train))
print("Unique values in y_val:", set(y_val))

from sklearn.preprocessing import LabelEncoder

# Fit encoder on all labels in training and validation sets
encoder = LabelEncoder()
y_train = encoder.fit_transform(y_train)
y_val = encoder.transform(y_val)

# Check the unique labels after encoding
print("Unique values in y_train after encoding:", set(y_train))
print("Unique values in y_val after encoding:", set(y_val))


Unique values in y_train: {0, 1, 2, 3, 4, 5, 6}
Unique values in y_val: {0, 1, 2, 3, 4, 5, 6}
Unique values in y_train after encoding: {0, 1, 2, 3, 4, 5, 6}
Unique values in y_val after encoding: {0, 1, 2, 3, 4, 5, 6}


In [13]:
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Embedding, LSTM, Dense

num_classes = len(set(y_train))  # Dynamically determine number of classes

model = Sequential([
    Embedding(input_dim=1000, output_dim=128, input_length=50),
    LSTM(128),
    Dense(num_classes, activation='softmax')  # Adjust num_classes here
])
model.compile(loss='sparse_categorical_crossentropy', optimizer='adam', metrics=['accuracy'])


/usr/local/lib/python3.11/dist-packages/keras/src/layers/core/embedding.py:90: UserWarning: Argument `input_length` is deprecated. Just remove it.
  warnings.warn(


In [14]:
import numpy as np

# Check the maximum and minimum values in your data
print("Max value in X_train_pad:", np.max(X_train_pad))
print("Min value in X_train_pad:", np.min(X_train_pad))
print("Max value in X_test_pad:", np.max(X_test_pad))
print("Min value in X_test_pad:", np.min(X_test_pad))

input_dim = np.max(X_train_pad) + 1  # Set input_dim to max index + 1
model = Sequential([
    Embedding(input_dim=input_dim, output_dim=128, input_length=50),
    LSTM(128),
    Dense(num_classes, activation='softmax')
])
model.compile(loss='sparse_categorical_crossentropy', optimizer='adam', metrics=['accuracy'])

# Clip values to be within the valid range
X_train_pad = np.clip(X_train_pad, 0, 999)
X_test_pad = np.clip(X_test_pad, 0, 999)

print("Max value in X_train_pad after fix:", np.max(X_train_pad))
print("Max value in X_test_pad after fix:", np.max(X_test_pad))


Max value in X_train_pad: 4999
Min value in X_train_pad: 0
Max value in X_test_pad: 4996
Min value in X_test_pad: 0
Max value in X_train_pad after fix: 999
Max value in X_test_pad after fix: 999


In [ ]:
history = model.fit(
    X_train_pad, y_train,
    validation_data=(X_test_pad, y_val),
    epochs=10,
    batch_size=32,
    verbose=1
)


Epoch 1/10
109/109 ━━━━━━━━━━━━━━━━━━━━ 14s 105ms/step - accuracy: 0.2942 - loss: 1.8343 - val_accuracy: 0.3238 - val_loss: 1.7930
Epoch 2/10
109/109 ━━━━━━━━━━━━━━━━━━━━ 9s 81ms/step - accuracy: 0.3057 - loss: 1.8122 - val_accuracy: 0.3238 - val_loss: 1.7914
Epoch 3/10
109/109 ━━━━━━━━━━━━━━━━━━━━ 12s 94ms/step - accuracy: 0.2976 - loss: 1.8173 - val_accuracy: 0.3238 - val_loss: 1.7877
Epoch 4/10
109/109 ━━━━━━━━━━━━━━━━━━━━ 21s 102ms/step - accuracy: 0.3014 - loss: 1.8065 - val_accuracy: 0.3238 - val_loss: 1.7863
Epoch 5/10
109/109 ━━━━━━━━━━━━━━━━━━━━ 21s 103ms/step - accuracy: 0.3077 - loss: 1.7999 - val_accuracy: 0.3238 - val_loss: 1.7891
Epoch 6/10
109/109 ━━━━━━━━━━━━━━━━━━━━ 19s 91ms/step - accuracy: 0.3049 - loss: 1.7971 - val_accuracy: 0.3238 - val_loss: 1.7888
Epoch 7/10
109/109 ━━━━━━━━━━━━━━━━━━━━ 10s 91ms/step - accuracy: 0.3110 - loss: 1.7952 - val_accuracy: 0.3238 - val_loss: 1.7879
Epoch 8/10
109/109 ━━━━━━━━━━━━━━━━━━━━ 11s 102ms/step - accuracy: 0.3155 - loss: 1.7957

In [15]:
import numpy as np
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score, classification_report
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Embedding, LSTM, Dense

# Split the data into 80% training and 20% validation
X_train, X_val, y_train, y_val = train_test_split(X_train_pad, y_train, test_size=0.2, random_state=42)

# Check the max and min values in training data
print("Max value in X_train:", np.max(X_train))
print("Min value in X_train:", np.min(X_train))
print("Max value in X_val:", np.max(X_val))
print("Min value in X_val:", np.min(X_val))

# Define input dimension
input_dim = np.max(X_train) + 1  # Set input_dim to max index + 1

# Define the model
model = Sequential([
    Embedding(input_dim=input_dim, output_dim=128, input_length=50),
    LSTM(128),
    Dense(len(set(y_train)), activation='softmax')  # Ensure correct number of output classes
])

# Compile the model
model.compile(loss='sparse_categorical_crossentropy', optimizer='adam', metrics=['accuracy'])

# Clip values to be within a valid range
X_train = np.clip(X_train, 0, 999)
X_val = np.clip(X_val, 0, 999)

# Train the model
history = model.fit(X_train, y_train, validation_data=(X_val, y_val), epochs=10, batch_size=32, verbose=1)

# Predict on validation data
y_val_pred_probs = model.predict(X_val)
y_val_pred = np.argmax(y_val_pred_probs, axis=1)  # Convert probabilities to class labels

# Compute accuracy
accuracy = accuracy_score(y_val, y_val_pred)
print(f"Validation Accuracy: {accuracy:.4f}")

# Compute F1-score
print("Classification Report:\n", classification_report(y_val, y_val_pred, digits=4))


Max value in X_train: 999
Min value in X_train: 0
Max value in X_val: 999
Min value in X_val: 0
Epoch 1/10
87/87 ━━━━━━━━━━━━━━━━━━━━ 19s 96ms/step - accuracy: 0.2886 - loss: 1.8415 - val_accuracy: 0.2999 - val_loss: 1.8009
Epoch 2/10
87/87 ━━━━━━━━━━━━━━━━━━━━ 10s 93ms/step - accuracy: 0.3257 - loss: 1.7901 - val_accuracy: 0.2999 - val_loss: 1.8089
Epoch 3/10
87/87 ━━━━━━━━━━━━━━━━━━━━ 11s 105ms/step - accuracy: 0.3214 - loss: 1.7861 - val_accuracy: 0.2999 - val_loss: 1.7990
Epoch 4/10
87/87 ━━━━━━━━━━━━━━━━━━━━ 7s 81ms/step - accuracy: 0.3090 - loss: 1.8015 - val_accuracy: 0.2999 - val_loss: 1.8077
Epoch 5/10
87/87 ━━━━━━━━━━━━━━━━━━━━ 11s 84ms/step - accuracy: 0.3155 - loss: 1.8027 - val_accuracy: 0.2999 - val_loss: 1.8028
Epoch 6/10
87/87 ━━━━━━━━━━━━━━━━━━━━ 9s 106ms/step - accuracy: 0.3121 - loss: 1.7979 - val_accuracy: 0.2984 - val_loss: 1.8167
Epoch 7/10
87/87 ━━━━━━━━━━━━━━━━━━━━ 7s 86ms/step - accuracy: 0.3167 - loss: 1.7853 - val_accuracy: 0.3013 - val_loss: 1.7716
Epoch 8/1

/usr/local/lib/python3.11/dist-packages/sklearn/metrics/_classification.py:1565: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))
/usr/local/lib/python3.11/dist-packages/sklearn/metrics/_classification.py:1565: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))
/usr/local/lib/python3.11/dist-packages/sklearn/metrics/_classification.py:1565: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))


In [19]:
import pandas as pd
import numpy as np
from sklearn.metrics import accuracy_score, classification_report
from tensorflow.keras.preprocessing.sequence import pad_sequences

# Load the test data (ensure it contains the 'content' and 'labels' columns)
test_df = pd.read_csv("/content/drive/MyDrive/PS_test.csv")

# Extract the text and labels
X_test = test_df['content'].values
y_test = test_df['labels'].values  # Actual labels from the test set

encoder = LabelEncoder()
y_test = encoder.fit_transform(y_test)
print(train_df['labels'].head(30))
print("y: ",y_test[0:30])
# Split into training and validation sets
#X_train, X_val, y_train, y_val = train_test_split(X, y, test_size=0.2, random_state=42)

# Convert labels to numerical values using the same encoder
#y_test = encoder.transform(y_test)

# Tokenize and pad test data
X_test_seq = tokenizer.texts_to_sequences(X_test)
X_test_pad = pad_sequences(X_test_seq, maxlen=50, padding='post', truncating='post')

# Clip values to be within valid range
X_test_pad = np.clip(X_test_pad, 0, 999)

# Predict on the test data
y_test_pred_probs = model.predict(X_test_pad)
y_test_pred = np.argmax(y_test_pred_probs, axis=1)  # Convert probabilities to class labels

# Compute accuracy
test_accuracy = accuracy_score(y_test, y_test_pred)
print(f"Test Accuracy: {test_accuracy:.4f}")

# Compute F1-score
print("Test Classification Report:\n", classification_report(y_test, y_test_pred, digits=4))


0               Neutral
1         Substantiated
2           Opinionated
3              Positive
4           Opinionated
5         Substantiated
6         Substantiated
7              Positive
8              Positive
9           Opinionated
10          Opinionated
11            Sarcastic
12          Opinionated
13             Positive
14            Sarcastic
15             Negative
16              Neutral
17            Sarcastic
18        Substantiated
19              Neutral
20              Neutral
21    None of the above
22          Opinionated
23              Neutral
24          Opinionated
25        Substantiated
26             Negative
27            Sarcastic
28            Sarcastic
29            Sarcastic
Name: labels, dtype: object
y:  [1 0 6 4 4 3 3 5 2 4 6 3 3 6 3 5 3 1 5 4 1 1 6 5 0 2 3 4 3 0]
17/17 ━━━━━━━━━━━━━━━━━━━━ 0s 24ms/step
Test Accuracy: 0.3254
Test Classification Report:
               precision    recall  f1-score   support

           0     0.0000    0.0000    0.0

/usr/local/lib/python3.11/dist-packages/sklearn/metrics/_classification.py:1565: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))
/usr/local/lib/python3.11/dist-packages/sklearn/metrics/_classification.py:1565: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))
/usr/local/lib/python3.11/dist-packages/sklearn/metrics/_classification.py:1565: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))


In [ ]:
print("Shape of X_test_pad:", X_test_pad.shape)
print("Number of rows in test_data:", len(test_data))

# Adjust X_test_pad to match the number of rows in test_data
X_test_pad = X_test_pad[:len(test_data)]

y_pred_probs = model.predict(X_test_pad)
y_pred = np.argmax(y_pred_probs, axis=1)


Shape of X_test_pad: (871, 50)
Number of rows in test_data: 544
17/17 ━━━━━━━━━━━━━━━━━━━━ 1s 22ms/step


In [ ]:
import pandas as pd
import numpy as np

# Define the mapping from numeric labels to textual descriptions
label_mapping = {
    0: "Negative",
    1: "Neutral",
    2: "None of the above",
    3: "Opinionated",
    4: "Positive",
    5: "Sarcastic",
    6: "Substantiated"
}

# 1. Load test data
test_data = pd.read_csv("/content/drive/MyDrive/PS_test_without_lables.csv")  # Replace with your test data file
print("Number of rows in test_data:", len(test_data))

# 2. Ensure X_test_pad matches the test data length
X_test_pad = X_test_pad[:len(test_data)]
print("Shape of X_test_pad after adjustment:", X_test_pad.shape)

# 3. Make predictions
y_pred_probs = model.predict(X_test_pad)  # Predicted probabilities
y_pred = np.argmax(y_pred_probs, axis=1)  # Convert probabilities to class labels

# 4. Map numeric labels to textual descriptions
y_pred_labels = [label_mapping[label] for label in y_pred]

# 5. Add predictions to test_data
test_data['Predicted_Label'] = y_pred_labels
test_data['Predicted_Probabilities'] = y_pred_probs.tolist()

# 6. Save results to CSV
output_file = "/content/drive/MyDrive/PS_test_predictions.csv"  # Specify the output path
test_data.to_csv(output_file, index=False)

print(f"Results saved to {output_file}")


Number of rows in test_data: 544
Shape of X_test_pad after adjustment: (544, 50)
17/17 ━━━━━━━━━━━━━━━━━━━━ 0s 23ms/step
Results saved to /content/drive/MyDrive/PS_test_predictions.csv
